In [1]:
from utils import *

INSTRUCTION = """
You are a game programmer for Pokemon. You will be given either of following two tasks.
1. I will give you a json script that details a pokemon role. Generate its python implementation.
2. I will give you a json script that details a pokemon role as well as its python implementation. This pokemon is gonna learn a new move or ability. Generate the incremental code based on the previous implementation. You can overload previous methods or add new ones.
""".strip()
INPUT1 = """
Role script:
```json
{}
```
""".strip()
INPUT2 = """
Role script:
```json
{}
```

Previous implementation:
```python
{}
```

{}
""".strip()
RESPONSE = """
```python
{}
```
""".strip()

In [2]:
def get_exs(entry, path1, path2):
    content = read("{}/{}.py".format(path1, entry))
    complete_script = read_json("{}/{}.json".format(path2, entry))
    move_list = list(complete_script["moves"].keys())
    ability_list = list(complete_script["ability"].keys())
    script = {
        "species": complete_script["species"],
        "types": complete_script["types"],
        "gender": complete_script["gender"],
        "ability": {ability_list[0]: complete_script["ability"][ability_list[0]]},
        "moves": {move_list[0]: complete_script["moves"][move_list[0]], move_list[1]: complete_script["moves"][move_list[1]]}
    }
    exs = []
    block = content.split("# -------------------------------------------------------------")[0].strip()
    ex = {"instruction": INSTRUCTION, "input": INPUT1.format(json.dumps(script, indent=2, ensure_ascii=False)), "response": RESPONSE.format(block)}
    exs += [ex]
    to_do = []
    if len(complete_script["ability"]) == 1:
        to_do = [{m: complete_script["moves"][m]} for m in move_list[2:]]
    elif len(complete_script["moves"]) == 5 and len(complete_script["ability"]) == 2:
        to_do = [{m: complete_script["moves"][m]} for m in move_list[2:-1]] + [{ability_list[-1]: complete_script["ability"][ability_list[-1]]}] + [{move_list[-1]: complete_script["moves"][move_list[-1]]}]
    else:
        to_do = [{m: complete_script["moves"][m]} for m in move_list[2:]] + [{ability_list[-1]: complete_script["ability"][ability_list[-1]]}]
    code = exs[-1]["response"].strip("```python\\n").strip()
    for d, block in zip(to_do, content.split("# -------------------------------------------------------------")[1:]):
        block = block.strip()
        is_ability = False if isinstance(list(d.values())[0], dict) else True
        new = "This role is gonna learn a new {}:\n```json\n{}\n```".format("ability" if is_ability else "move", json.dumps(d, indent=2, ensure_ascii=False))
        ex = {"instruction": INSTRUCTION, "input": INPUT2.format(json.dumps(script, indent=2, ensure_ascii=False), code, new), "response": RESPONSE.format(block)}
        exs += [ex]
        if is_ability:
            script["ability"].update(d)
        else:
            script["moves"].update(d)
        code = "\n\n".join([code, exs[-1]["response"].strip("```python\\n").strip()])
    return exs

In [3]:
exs = []
n = 0
for filename in os.listdir("data/train/code"):
    if not filename.endswith(".py"): continue
    entry = filename.split(".")[0]
    exs += get_exs(entry, "data/train/code", "data/train/script")
    n += 1
write_jsonl(exs, "train.jsonl")
n, len(exs)

(170, 480)